### Tests for Applying Gains Matrix

In [1]:
import sys
from pathlib import Path
parent_dir = Path.cwd().parent
sys.path.insert(0, str(parent_dir))
print(parent_dir)

/home/mike/corrcal_gpu_pipeline/pipeline


In [2]:
import cupy as cp
import numpy as np
from utils import *
from zp_puregpu_funcs_py import *
from invcov import *

/home/mike/corrcal_gpu_pipeline/pipeline/utils.py:119: SyntaxWarning: invalid escape sequence '\D'
  """


hera_cal is not installed. Some gridding features unavailable.
pyfof is not installed. Some gridding features unavailable.


In [3]:
apply_gains_to_mat?
# undo_zeropad?

In [4]:
summarize_benchmark_results?

In [5]:
#simulation params relevant for testing application of gains to a matrix
n_ant = 4
n_bl = 30
n_gains = 2*n_bl
n_eig = 3
xp = cp  #run things on the gpu using cupy

#this might be the easiest (and most general) way to devise an edges
# array, though we hard code an ex. edges array to be sure it fits
# the desired format of having no odd entries.
edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = n_ant)*2))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
xp = cp
sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
ant_1_array = cp.arange(n_bl//2)
ant_2_array = cp.arange(n_bl//2)

#zeropad the noise, diff, source mats
zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

#Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)

The edges of the redundant blocks have indices[ 0  8 10 14 20 30]


In [6]:
applied_gains = apply_gains_to_mat(sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_1_array, cp, True)

In [7]:
print(f"Input mat has shape {sim_diff_mat_3d.shape}")
print()
print(f"Gains applied mat has shape {applied_gains.shape}")
print()
print(f"The mat with gains applied looks like:\n\n{applied_gains}")

Input mat has shape (5, 10, 3)

Gains applied mat has shape (5, 10, 3)

The mat with gains applied looks like:

[[[0.27814241 0.25826757 0.11502817]
  [0.01206056 0.10694945 0.26610058]
  [0.07658128 0.034721   0.01068864]
  [0.09484834 0.04758766 0.11834722]
  [1.15514125 0.40169004 0.49273793]
  [0.7461802  1.05070825 0.8775721 ]
  [0.19946181 0.04694314 0.22082005]
  [0.17122408 0.22419827 0.07955064]
  [0.         0.         0.        ]
  [0.         0.         0.        ]]

 [[0.55076088 0.58071516 0.00163646]
  [0.37432234 0.15166256 0.05861349]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]]

 [[0.34896338 0.32980002 0.10796866]
  [0.01199912 0.02940281 0.26329288]
  [0.73105055 1.10113589 0.0054533 ]
  [1.18148322 0.78116352 1.190657

#### Random Debugging tests

In [8]:
m = np.random.rand(2, 5, 3)
g = np.random.rand(10)

gs = g[:, None] * g[:, None]
gs = gs.reshape(m.shape[0], m.shape[1], 1)

print(gs.shape)
print(m.shape)
print(g.shape)
print(m)
print(m[:, ::2])
out = gs[:, ::2] * m[:, ::2]

(2, 5, 1)
(2, 5, 3)
(10,)
[[[0.63475242 0.47749868 0.56849862]
  [0.8918816  0.26439803 0.22116261]
  [0.61738785 0.56877546 0.86717425]
  [0.83886075 0.16462868 0.16543123]
  [0.36727472 0.99467814 0.08296931]]

 [[0.8536953  0.16457528 0.38804363]
  [0.19823186 0.34731467 0.28966187]
  [0.45115672 0.09476534 0.32697205]
  [0.04419674 0.26274277 0.25307845]
  [0.01064217 0.36801719 0.26066237]]]
[[[0.63475242 0.47749868 0.56849862]
  [0.61738785 0.56877546 0.86717425]
  [0.36727472 0.99467814 0.08296931]]

 [[0.8536953  0.16457528 0.38804363]
  [0.45115672 0.09476534 0.32697205]
  [0.01064217 0.36801719 0.26066237]]]


In [9]:
a = np.arange(5)
b = np.arange(5,10)
c = np.random.rand((a.shape[0] + b.shape[0]))
print(c)
d = c[a]
e = c[b]

print(d)
print()
print(e)
print()
print(np.concatenate((d, e)))
print()
aa = np.concatenate((d, e))

print(aa[:int(len(aa)/2)])

print(aa[int(len(aa)/2):])

[0.12619605 0.91773981 0.50993003 0.81121522 0.84217708 0.8871101
 0.66440163 0.69356656 0.96790111 0.18823812]
[0.12619605 0.91773981 0.50993003 0.81121522 0.84217708]

[0.8871101  0.66440163 0.69356656 0.96790111 0.18823812]

[0.12619605 0.91773981 0.50993003 0.81121522 0.84217708 0.8871101
 0.66440163 0.69356656 0.96790111 0.18823812]

[0.12619605 0.91773981 0.50993003 0.81121522 0.84217708]
[0.8871101  0.66440163 0.69356656 0.96790111 0.18823812]


#### Check ZP computes the same thing as corrcal

In [10]:
#Current corrcal gain application function
def apply_gains_to_mat_corrcal(gains, mat, ant_1_array, ant_2_array):
    """Apply a gain-like matrix to a provided matrix."""
    complex_gains = gains[::2] + 1j*gains[1::2]
    gain_mat = (
        complex_gains[ant_1_array,None] * complex_gains[ant_2_array,None].conj()
    )
    out = np.zeros_like(mat)
    out[::2] = gain_mat.real * mat[::2] - gain_mat.imag * mat[1::2]
    out[1::2] = gain_mat.imag * mat[::2] + gain_mat.real * mat[1::2]
    return out

In [11]:
#simulation params relevant for testing application of gains to a matrix
n_ant = 15
n_bl = 2*n_ant
n_gains = 4*n_ant
n_eig = 3
xp = cp  #run things on the gpu using cupy

#this might be the easiest (and most general) way to devise an edges
# array, though we hard code an ex. edges array to be sure it fits
# the desired format of having no odd entries.
edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = 3)*2))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
xp = cp
sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
ant_1_array = cp.arange(n_ant)
ant_2_array = cp.arange(n_ant, 2*n_ant)

#zeropad the noise, diff, source mats
zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

#Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)


The edges of the redundant blocks have indices[ 0  6  8 18 30]


In [12]:
out_corrcal = apply_gains_to_mat_corrcal(sim_gains, sim_diff_mat, ant_1_array, ant_2_array)
out_gpu = apply_gains_to_mat(sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_2_array, cp)
out_gpu_corrcal = apply_gains_to_mat(sim_gains, sim_diff_mat, edges, ant_1_array, ant_2_array, cp, False)
out_gpu_resh = undo_zeropad(out_gpu, edges, cp)

In [13]:
#should return True if my routine computes the same thing as the current corrcal routine
print(cp.allclose(out_gpu_resh, out_gpu_corrcal))
print(cp.allclose(out_gpu_resh, out_corrcal))

True
True


#### Timing test for np vs cp versions

In [14]:
#simulation params relevant for testing application of gains to a matrix
n_bl = 240000  # something like 2*N_ant^2 for Re/Im split
n_ant = 512
n_gains = 2*n_bl  #one for each antenna and one for each Re and Im part
n_eig = 3
xp = cp  #run things on the gpu using cupy

#this might be the easiest (and most general) way to devise an edges
# array, though we hard code an ex. edges array to be sure it fits
# the desired format of having no odd entries.
if xp == cp:

    edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = n_ant)*2))
    edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
    print(f"The edges of the redundant blocks have indices{edges}")

    #some random noise, diffuse, and source covariance matrices
    sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
    sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant

    #leave as n_bl/2 since indexex into cplex gains which are not Re/Im split
    ant_1_array = cp.arange(n_bl//2) 
    ant_2_array = cp.arange(n_bl//2)

    #zeropad the noise, diff, source mats
    zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

    #Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
    sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)

elif xp == np:
    edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = n_ant)*2))
    edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
    print(f"The edges of the redundant blocks have indices{edges}")

    #some random noise, diffuse, and source covariance matrices
    sim_diff_mat = xp.random.rand(n_bl, n_eig).astype('float64')
    sim_gains = xp.random.rand(n_gains).astype('float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
    ant_1_array = xp.arange(n_bl//2)
    ant_2_array = xp.arange(n_bl//2)

    #zeropad the noise, diff, source mats
    # zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

    #Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
    # sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)

print(len(edges))

The edges of the redundant blocks have indices[     0   1204   1736   2320   2628   4564   4876   5128   5928   6620
   7876   8192   8850   9402   9616   9844   9964  10078  10418  12312
  12596  13346  13494  13810  14142  14426  15456  15838  15978  16388
  16682  16810  17308  17870  17976  18346  18354  19212  19236  20080
  20436  20660  21146  21158  22258  22292  24528  25070  27146  27866
  28206  28696  28908  30602  31924  32076  33320  33612  34768  34790
  35278  35298  35590  36006  36058  36440  36480  36782  36878  36884
  36982  37892  37928  38582  39310  39702  42084  42260  42566  44028
  44460  44834  44884  46000  47308  47684  47728  47796  47852  47902
  48148  49052  49420  49494  49938  50796  51980  52084  52646  52742
  52878  52936  55066  55562  55666  55760  56596  57010  57394  58404
  58568  60386  60730  60950  62220  62380  62840  63180  64416  65516
  66008  66068  66344  66834  67158  67174  67182  67544  67562  68014
  68658  68710  70172  71380  7

In [15]:
print(type(sim_diff_mat))
print(type(sim_gains))
summarize_benchmark_results(apply_gains_to_mat, sim_gains, sim_diff_mat, edges, ant_1_array, ant_2_array, xp, False)

<class 'cupy.ndarray'>
<class 'cupy.ndarray'>
Time on cpu: 0.001249s
Time on gpu: 0.001285s


In [16]:
print(type(sim_diff_mat))
print(type(sim_gains))
summarize_benchmark_results(apply_gains_to_mat, sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_2_array, xp, True)

<class 'cupy.ndarray'>
<class 'cupy.ndarray'>
Time on cpu: 0.002612s
Time on gpu: 0.003781s


### Random debugging....

In [17]:
import numpy as np
mat = np.random.rand(3, 2, 3)
print(mat)
gm = np.arange(6)
gm = gm.reshape(3, 2, 1)
print(gm)
print()
print(gm * mat)

[[[0.85721108 0.23689334 0.42003565]
  [0.90606059 0.82252577 0.65270619]]

 [[0.26255225 0.41913522 0.25920534]
  [0.18046729 0.32509769 0.24629263]]

 [[0.60456372 0.30078803 0.76859114]
  [0.30221224 0.65796965 0.32786387]]]
[[[0]
  [1]]

 [[2]
  [3]]

 [[4]
  [5]]]

[[[0.         0.         0.        ]
  [0.90606059 0.82252577 0.65270619]]

 [[0.52510451 0.83827045 0.51841068]
  [0.54140188 0.97529306 0.7388779 ]]

 [[2.41825487 1.20315212 3.07436455]
  [1.51106122 3.28984825 1.63931933]]]


In [18]:
a = np.random.rand(4,4)
print(a)
print(a[::2])

[[0.37221298 0.04985606 0.76610506 0.2591707 ]
 [0.80122092 0.20924899 0.1499741  0.69911432]
 [0.01794594 0.51785232 0.8381936  0.45197051]
 [0.07358859 0.61304657 0.4708645  0.18181427]]
[[0.37221298 0.04985606 0.76610506 0.2591707 ]
 [0.01794594 0.51785232 0.8381936  0.45197051]]


In [19]:
a = np.arange(15)
b = np.arange(15)

print(a[:, None]* b[:, None])

[[  0]
 [  1]
 [  4]
 [  9]
 [ 16]
 [ 25]
 [ 36]
 [ 49]
 [ 64]
 [ 81]
 [100]
 [121]
 [144]
 [169]
 [196]]
